In [5]:
text = open("input.txt", "r").read()
print("total number of chars : ", len(text))


total number of chars :  1115394


In [6]:
print(text[:1000])


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [7]:
# build vocab
chars = sorted(list(set("".join(text))))

vocab_size = len(chars)
print("vocab size : ", vocab_size)

vocab size :  65


In [9]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for ch, i in stoi.items()}

# take string as input and return array of integers
encode = lambda s : [stoi[ch] for ch in s]

# take list of integers and return a string
decode = lambda l : "".join([itos[i] for i in l])

e = encode("aditya")
print(e)
d = decode(e)
print(d)

[39, 42, 47, 58, 63, 39]
aditya


In [10]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [11]:
# split the data into train, test, split

n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [12]:
block_size = 9
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])

In [13]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("when input is : ", context, " | The target is : ", target)

when input is :  tensor([18])  | The target is :  tensor(47)
when input is :  tensor([18, 47])  | The target is :  tensor(56)
when input is :  tensor([18, 47, 56])  | The target is :  tensor(57)
when input is :  tensor([18, 47, 56, 57])  | The target is :  tensor(58)
when input is :  tensor([18, 47, 56, 57, 58])  | The target is :  tensor(1)
when input is :  tensor([18, 47, 56, 57, 58,  1])  | The target is :  tensor(15)
when input is :  tensor([18, 47, 56, 57, 58,  1, 15])  | The target is :  tensor(47)
when input is :  tensor([18, 47, 56, 57, 58,  1, 15, 47])  | The target is :  tensor(58)
when input is :  tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])  | The target is :  tensor(47)


In [19]:
torch.manual_seed(1337)
batch_size = 4  # basic batch calculation
block_size = 8  # how many set of character I am looking further

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    return x, y


xb, yb = get_batch("train")
print("inputs")
print(xb.shape)
print(xb)
print("targets")
print(yb.shape)
print(yb)

print("-----")

for b in range(batch_size):  # batch dimentions
    for t in range(block_size): # time dimention
        context = xb[b, :t]
        target = yb[b, t]

        print("when the input is : ", context, " | The target is : ", target)

    print()
    


inputs
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----
when the input is :  tensor([], dtype=torch.int64)  | The target is :  tensor(43)
when the input is :  tensor([24])  | The target is :  tensor(58)
when the input is :  tensor([24, 43])  | The target is :  tensor(5)
when the input is :  tensor([24, 43, 58])  | The target is :  tensor(57)
when the input is :  tensor([24, 43, 58,  5])  | The target is :  tensor(1)
when the input is :  tensor([24, 43, 58,  5, 57])  | The target is :  tensor(46)
when the input is :  tensor([24, 43, 58,  5, 57,  1])  | The target is :  tensor(43)
when the input is :  tensor([24, 43, 58,  5, 57,  1, 46])  | The target i

In [67]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()

        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx)    # (B, T, C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            # logits = logits.view(B*T, C)
            # targets = targets.view(B*T)
            logits = torch.permute(logits, (0, 2, 1))
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        out = []
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply the softmax to get the probabilities
            probs = F.softmax(logits, dim=1)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
            idx = idx_next
            out.append(idx.item())
        return out

            
    

m = BigramLanguageModel(vocab_size)
out, loss = m(xb, yb)
print("vocab size : ", vocab_size)
print("input shape : ", xb.shape)
print("output shape : ", yb.shape)
print(out.shape)

print("loss : ", loss.item())

idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)))


vocab size :  65
input shape :  torch.Size([32, 8])
output shape :  torch.Size([32, 8])
torch.Size([32, 65, 8])
loss :  4.80723237991333
SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [68]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)



In [74]:
batch_size = 32

for steps in range(10000):
    # sample a batch of data
    xb, yb = get_batch("train")

    # evaluate the loss
    logits, loss = m(xb, yb)
    # print(logits.shape)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    # break

print(loss.item())

2.386481523513794


In [79]:
idx = torch.zeros((1,1), dtype=torch.long)

print(decode(m.generate(idx, max_new_tokens=400)))


She t chaventhand gend s; CI co; inut'se!
JUn anounde g IVIfotto me h enkedoudin. sers, fofote bicar me g
oull oupasthosthane oforurees nd dovey,

T:
Fosy,

A:
s, courerd whouthoreacce!
Tho?'sto arend heshen't ur ks mearadavize fl nks k sck br, ro to y'ded:
PUpusother.

YO:

CLIfevid at ner,


ETot y te se mstotrcon te ly ors nrewir INTwn ico rs Harl t thetck, totu as, lithanthetheser bls ou titow


# Mathematical trick in self attettion

In [3]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2 # batch, time, channels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [4]:
xbow = torch.zeros((B, T, C))

for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t, C)
        xbow[b, t] = torch.mean(xprev, 0)

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

#single head of self attention
head_size = 16
key = nn.Linear(C, head_size, bias=False) # (C, 16)
query = nn.Linear(C, head_size, bias = False) # (C, 16)
value = nn.Linear(C, head_size, bias=False) #(C, 16)

k = key(x)  # (B, T, 16)
q = query(x) # (B, T, 16)

wei = q @ k.transpose(-2, -1)   # (B, T, 16) @ (B, 16, T) -> # (B, T, T)
tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=-1)
v = value(x)
out = wei @ v

out.shape


torch.Size([4, 8, 16])

In [13]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.8217, 0.1783, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2452, 0.5850, 0.1698, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1719, 0.6665, 0.0966, 0.0650, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1250, 0.7702, 0.0514, 0.0275, 0.0258, 0.0000, 0.0000, 0.0000],
        [0.0512, 0.0047, 0.1479, 0.3085, 0.4866, 0.0011, 0.0000, 0.0000],
        [0.1432, 0.1495, 0.1443, 0.1454, 0.1318, 0.1407, 0.1452, 0.0000],
        [0.0857, 0.1450, 0.0583, 0.0440, 0.0667, 0.3253, 0.0829, 0.1921]],
       grad_fn=<SelectBackward0>)